In [1]:
from shapely import Point
from tqdm import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [2]:
shapefile = gpd.read_file("./Shapefiles/Breda.shp")

green = pd.read_csv("./Datasets/Green.csv")
income = pd.read_csv("./Datasets/Income.csv")

In [3]:
def identify_neighborhood(lon, lat):
    point = Point(lon, lat)
    for index, feature in shapefile.iterrows():
        if feature.geometry.contains(point):
            return feature["BUURT"]
    return "Out of Bounds"

def calculate_average_green_index(neighborhood):
    if neighborhood != "Out of Bounds":
        neighborhood_green = green.loc[green["Neighborhood"] == neighborhood]
        if not neighborhood_green.empty:
            return neighborhood_green["Green Score"].mean()
    return None

In [4]:
tqdm.pandas(desc = "Splitting coordinates...")
green[["Longitude", "Latitude"]] = green[["Longitude", "Latitude"]].progress_apply(lambda x: x.str.split(","))

tqdm.pandas(desc = "Converting to numeric...")
green[["Longitude", "Latitude"]] = green[["Longitude", "Latitude"]].progress_apply(lambda x: pd.to_numeric(x.str[0] + "." + x.str[1]))

tqdm.pandas(desc = "Identifying neighborhoods...")
green["Neighborhood"] = green.progress_apply(lambda row: identify_neighborhood(row["Longitude"], row["Latitude"]), axis = 1)

Identifying neighborhoods...: 100%|██████████| 298038/298038 [03:43<00:00, 1336.37it/s]


In [5]:
averages = income.copy()
averages[["Neighborhood", "Income"]] = averages["Neighborhood;Income"].str.split(";", expand = True)
averages = averages.drop(columns = ["Neighborhood;Income"])
averages["Income"] = pd.to_numeric(averages["Income"], errors = "coerce")

min_income, max_income = averages["Income"].min(), averages["Income"].max()
averages["Normalized Income"] = (averages["Income"] - min_income) / (max_income - min_income)

print(averages.head())
averages["Average Green Index"] = averages.apply(lambda row: calculate_average_green_index(row["Neighborhood"]), axis = 1)

min_green, max_green = averages["Average Green Index"].min(), averages["Average Green Index"].max()
averages["Normalized Green Index"] = (averages["Average Green Index"] - min_green) / (max_green - min_green)

merged = shapefile.merge(averages, left_on = "BUURT", right_on = "Neighborhood", how = "left")

correlation = merged[["Normalized Green Index", "Normalized Income"]].corr().iloc[0, 1]

merged["Controlled Green Index"] = merged["Normalized Green Index"] - (correlation * merged["Normalized Income"])

min_controlled_green = merged["Controlled Green Index"].min()
max_controlled_green = merged["Controlled Green Index"].max()

merged["Relative Difference"] = merged["Normalized Green Index"] - merged["Controlled Green Index"]

merged["Controlled Green Index"].fillna(0, inplace = True)
merged["Normalized Controlled Green Index"] = (merged["Controlled Green Index"] - min_controlled_green) / (max_controlled_green - min_controlled_green)

  Neighborhood  Income  Normalized Income
0         City       2           0.333333
1   Valkenberg       2           0.333333
2       Chasse       2           0.333333
3   Fellenoord       1           0.000000
4   Schormolen       1           0.000000


KeyError: ('Normalized Income', 'Normalized Green Index', 'Relative Difference', 'Controlled Green Index')

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 1, ncols = 4, figsize = (24, 6))

merged.plot(
    column = "Normalized Income",
    cmap = "Blues",
    linewidth = 0.8,
    ax = ax1,
    edgecolor = "Black",
    legend = True,
    vmin = 0,
    vmax = 1
)
ax1.set_title("Average Income")

merged.plot(
    column = "Normalized Green Index",
    cmap = "Greens",
    linewidth = 0.8,
    ax = ax2,
    edgecolor = "Black",
    legend = True,
    vmin = 0,
    vmax = 1
)
ax2.set_title("Average Green Score")

merged.plot(
    column = "Controlled Green Index",
    cmap = "Greens",
    linewidth = 0.8,
    ax = ax3, 
    edgecolor = "Black",
    legend = True,
    vmin = 0,
    vmax = 1
)
ax3.set_title("Green Index Controlled for Income")

merged.plot(
    column = "Relative Difference",
    cmap = "Reds",
    linewidth = 0.8,
    ax = ax4,
    edgecolor = "Black",
    legend = True,
    vmin = 0,
    vmax = 1
)
ax4.set_title("Uncontrolled vs Controlled Difference")

ax1.set_axis_off()
ax2.set_axis_off()
ax3.set_axis_off()
ax4.set_axis_off()

plt.show()